<a href="https://colab.research.google.com/github/27abernal/Adv_AI/blob/main/MNIST_NN_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -----------------------------
# MNIST DIGIT CLASSIFIER (PyTorch)
# -----------------------------

# Import the main PyTorch library (tensors, automatic differentiation, etc.)
import torch

# Import the neural network module (contains layers, loss functions, etc.)
import torch.nn as nn

# Import PyTorch's optimization algorithms (SGD, Adam, etc.)
import torch.optim as optim

# Import utilities for loading common datasets and applying transformations
from torchvision import datasets, transforms

# Import DataLoader to batch, shuffle, and load data efficiently
from torch.utils.data import DataLoader

# in order to complete Part 6, drawing out a number
import gradio as gr
from PIL import ImageOps, Image

In [2]:
# -----------------------------
# 1. LOAD DATA
# Transforms are preprocessing steps that get applied automatically to every image
# you load from a dataset.
# Think of transforms as a recipe that says:

# “Every time you give me an image, do X, then Y, then Z to it.”
# “For every MNIST image: convert it to a PyTorch tensor.
# MNIST images come in as PIL images (Python Imaging Library).
# But your neural network expects tensors.

# For MNIST specifically:
# - The raw images are stored as PIL images (Python Imaging Library format).
# - Neural networks in PyTorch expect input in the form of tensors.
# - transforms.ToTensor() converts a PIL image (with pixel values 0–255)
#   into a PyTorch tensor with values scaled to the range 0–1.

# transforms.Compose([...]) lets you chain together multiple preprocessing steps.
# Here we only apply one step, but you could add things like normalization,
# data augmentation, or reshaping if needed.
# -----------------------------
transform = transforms.Compose([
    transforms.ToTensor() # Convert PIL image → PyTorch FloatTensor (Channel x Height x Width), with pixel values scaled to [0, 1]
])

In [3]:
# Load training dataset (MNIST)
train_dataset = datasets.MNIST(
    root="./data",        # Folder where the MNIST data will be stored. If it doesn't exist, PyTorch will create it.
    train=True,           # Specifies that we want the training portion of the MNIST dataset (60,000 images).
                          # If False, we would get the test set instead (10,000 images).
    transform=transform,  # Apply the preprocessing steps we defined earlier.
                          # Every image will automatically be converted to a PyTorch tensor and scaled to [0, 1].
    download=True         # If the dataset isn't already in the "data" folder, PyTorch will download it automatically.
)


In [4]:
# Load test dataset (see the comments for the training dataset)
test_dataset = datasets.MNIST(
    root="./data",
    train=False,
    transform=transform,
    download=True
)


In [5]:
# Make DataLoaders, which wrap the dataset and handle batching, shuffling, and loading data efficiently.
# a batch is a small group of training examples processed together at once.

# for the training dataset, the batch size is 32 and data will be shuffled every epoch to improve training
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# for the testing dataset, the batch size is 32 and data will not be shuffled as evaluation should remain consistent
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

# TODO: Access and print the unique labels in the training data set using the train_loader object
# Create an empty set to store the unique labels (digits 0–9). A set automatically removes duplicates.
unique_labels = set()

# Loop through each batch of data in the training DataLoader.
# Each batch gives: images in a tensor of shape (batch_size, 1, 28, 28), and labels in a tensor of shape (batch_size) containing digit labels
for images, labels in train_loader:
    # Convert the label tensor to a regular list, then add all labels in the list to the set.
    # .update() adds multiple items at once.
    unique_labels.update(labels.tolist())

# Print the set of labels found in the training dataset.
print("Unique labels in training data:", unique_labels)

Unique labels in training data: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}


In [6]:
# -----------------------------
# 2. DEFINE NEURAL NETWORK
# TODO: Design a Neural Network with 1 hidden layer of 128 neurons
# -----------------------------
class SimpleNN(nn.Module):
    def __init__(self):
        super().__init__()           # Initialize the parent nn.Module
        # TODO: Define layers
        self.fc1 = nn.Linear(28*28, 128)  # Input → hidden
        self.fc2 = nn.Linear(128, 128)    # Hidden → hidden
        self.fc3 = nn.Linear(128, 10)     # Hidden → output

    def forward(self, x):
        # Flatten image: (batch, 1, 28, 28) → (batch, 784)
        x = x.view(-1, 28*28)
        x = torch.relu(self.fc1(x))       # Hidden layer + ReLU activation
        x = torch.relu(self.fc2(x))       # Hidden layer + ReLU activation
        x = self.fc3(x)                   # Output layer (logits)
        return x


In [7]:
# TODO: Create the model
model = SimpleNN()  # Instantiate the neural network


In [8]:
# -----------------------------
# 3. LOSS FUNCTION + OPTIMIZER
# -----------------------------
# TODO: Define your loss function
criterion = nn.CrossEntropyLoss()  # Computes loss between predicted logits and true labels

# TODO: Setup your gradient descent. Try different values for the learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
# Uses stochastic (batched) gradient descent to update model weights
# lr=0.01 sets the learning rate (step size for updates)

In [9]:
# -----------------------------
# 4. TRAINING LOOP
# -----------------------------

# TODO: Define the number of epochs
epochs = 15  # Number of times the model will see the entire training set

for epoch in range(epochs):
    model.train()          # Set model to training mode
    total_loss = 0         # Initialize loss accumulator for this epoch

    for images, labels in train_loader:
        # TODO: Reset the gradients
        optimizer.zero_grad()  # Clear old gradients before this step

        # TODO: Forward pass
        outputs = model(images)  # Compute predictions for this batch

        # TODO: Compute loss
        loss = criterion(outputs, labels)  # Compare predictions with true labels using the defined loss function

        # TODO: Backpropagate
        loss.backward()        # Compute gradients of loss with respect to model parameters

        # TODO: Update gradients
        optimizer.step()       # Adjust model parameters using the gradients

        total_loss += loss.item()  # Add this batch's loss to the total

    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")


Epoch 1, Loss: 2067.8470
Epoch 2, Loss: 702.4889
Epoch 3, Loss: 577.9131
Epoch 4, Loss: 503.6303
Epoch 5, Loss: 446.0313
Epoch 6, Loss: 396.9026
Epoch 7, Loss: 357.0687
Epoch 8, Loss: 322.8693
Epoch 9, Loss: 294.1726
Epoch 10, Loss: 269.5456
Epoch 11, Loss: 247.4395
Epoch 12, Loss: 229.5689
Epoch 13, Loss: 213.5006
Epoch 14, Loss: 198.4258
Epoch 15, Loss: 185.5698


In [10]:
# -----------------------------
# 5. EVALUATION
# -----------------------------
correct = 0              # Initialize counter for correct predictions
total = 0                # Initialize counter for total samples
model.eval()             # Set model to evaluation mode (disables dropout, batch norm updates)

with torch.no_grad():
    for images, labels in test_loader:
        # TODO: Forward pass
        outputs = model(images)

        # Predicted class = index of max logit
        _, predicted = torch.max(outputs.data, 1)  # Get class with highest score for each image

        total += labels.size(0)          # Increment total by batch size just processed
        correct += (predicted == labels).sum().item()  # Count correct predictions

print(f"Test Accuracy: {100 * correct / total:.2f}%")


Test Accuracy: 96.80%


In [11]:
# -----------------------------
# 6. TEST SINGLE PREDICTION
# ------------------------------
# Gradio Sketchpad gives you:
# * a full-color NumPy array
# * black digit on white background
# * large resolution
# * no consistent scale
# Hence the preprocessing
# ------------------------------

def preprocess_image(image):
    sketch_transform = transforms.Compose([
    transforms.ToPILImage(),                      # NumPy → PIL
    transforms.Grayscale(),                       # ensure 1 channel
    transforms.Resize((28, 28)),                  # 28x28 like MNIST
    transforms.Lambda(lambda img: ImageOps.invert(img)),  # invert colors
    transforms.ToTensor(),                        # → tensor, shape (1,28,28), values in [0,1]
    ])
    # Gradio Sketchpad sometimes passes a dict with 'composite'
    if isinstance(image, dict):
        image = image['composite']   # this is a NumPy array

    # Apply the preprocessing transform
    img_tensor = sketch_transform(image)  # (1, 28, 28)

    # Add batch dimension → (1, 1, 28, 28)
    img_tensor = img_tensor.unsqueeze(0)

    return img_tensor

def predict_digit(image):
    # --- STEP 1: CHECK IF SOMETHING HAS BEEN DRAWN ---
    if image is None: return "Draw something!"

    # --- STEP 2: PREPROCESS THE IMAGE ---
    img_tensor = preprocess_image(image)

    # --- STEP 3: RUN THE MODEL ---
    with torch.no_grad():
        prediction = model(img_tensor)

        # Get the index of the highest score (the predicted digit)
        predicted_digit = torch.argmax(prediction).item()

    return str(predicted_digit)

# UI Setup
interface = gr.Interface(fn=predict_digit, inputs=gr.Sketchpad(label="Draw Here"), outputs="label")
interface.queue().launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://219fb9fbce6c93650d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
